# Mashup datasets

This Jupyter Notebook analyses the mashup datasets for ["Blessed Be the Fruit"](https://github.com/OrsolaMBorrini/blessedfruit), an open data project regarding the analysis of factors that might influence pregnancy rates in young women in Italy.

From the starting 7 source datasets, three different mashup datasets have been created:
Id | Dataset | Description (factor of interest) | Original source datasets
--- | --- | --- | --- 
MD1 | Religious observance in each region | RELIGION - % of religious observance in each region (over the total population) | D1, D2, D3 
MD2 | Pregnancy rates in young women in each region | PREGNANCY - % of pregnancies in young women (15-25) in each region (over the total population of young women aged 15-25) | D4, D5, D6
MD3 | (Higher) education rates in young women in each region | EDUCATION - % of women early leavers (18-24) in each region (over the total population) | D1, D2, D7

### Disclaimer
This Jupyter Notebook is of informational nature only, it is not thought to be used for the clean up process for the datasets, but only for their analysis and the explanation of the process. The code actually used for the clean up can be found in `scripts > MASHUP.py`.


## Mashup phase
### MD1 - Religious observance in each region
To create a dataset showing the percentage of religious observance in each region, we need to compare the total general population of each region (with no distinction of gender) to the religious observance (in thousands value) in each region. To get this result we will use the following source datasets:
* D1 and D2 -> for the data regarding total general population in each region for each of the three years of interest (`data > cleanDS > Population2017_Generalclean.csv`, `Population2018_Generalclean.csv`, `Population2019_Generalclean.csv`)
* D3 -> for the data regarding religious observance (thousands value) in each region (`data > cleanDS > D3_clean.csv`)


In [1]:
import csv
from pandas import *

md1 = DataFrame()

d3_clean = read_csv("../data/cleanDS/D3_clean.csv")
d3_clean

,ITTER107,Territory,TIPO_DATO_AVQ,TIME,Value
0,ITD3,Veneto,6_WEEK_RELIG,2017,1302
1,ITD3,Veneto,6_WEEK_RELIG,2018,1155
2,ITD3,Veneto,6_WEEK_RELIG,2019,1129
3,ITC4,Lombardia,6_WEEK_RELIG,2017,2539
4,ITC4,Lombardia,6_WEEK_RELIG,2018,2371
...,...,...,...,...,...
115,ITE1,Toscana,6_NEVER_RELIG,2018,1308
116,ITE1,Toscana,6_NEVER_RELIG,2019,1368
117,ITF2,Molise,6_NEVER_RELIG,2017,51
118,ITF2,Molise,6_NEVER_RELIG,2018,56


### MD2 - Pregnancy rates in young women in each region

### MD3 - (Higher) education rates in young women in each region
As the source dataset D7 did not have, with absolute values, the subdivision in regions (but only had data regarding the whole national territory of Italy), we had to use the "percentage values".
The percentage is calculated on the total population of 18-24 years old in each region, without distinctions in terms of gender.
However, given the assumptions of the project, we had to focus on the female population of this age range and we did so by employing the following proportion:
* % early leavers : total 18-24 = [% early leavers F] : total 18-24 F
We did so for each region and therefore gathered the data concerning only females aged 18-24.